# Daily Informer
> This script sends a message everyday including iunformation about sports and temas I follow.

## Imports 
> Im using pandas this time for convenience 

In [ ]:
!pip install beautifulsoup4 imgkit html2image 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime as dt
from IPython.core.display import HTML
import dateutil
import re


## Templates

In [ ]:
def start_template():
  start_template = '''
<html>
    <head>
        <title>Daily Report</title>
    </head>
    <body>
          
          '''
  return start_template



def f1_header_template():
  f1_header = '''<div style=" background: url('https://www.hdcarwallpapers.com/download/ferrari_f1_75_2022_formula_1_5k_2-1920x1080.jpg') no-repeat center center; text-align: center; height: 300px; margin-top: 0 px;  text-align: center; border: 3px solid green">
  <div style="margin-top: 100px;">
    <h1><strong> Formula 1 Standings and incoming event</strong></h1>
  </div>
</div>'''
  return f1_header



def end_template():
  end_template= '''<style>
          .dataframe {
                  margin-left: auto;
                  margin-right: auto; }
          </style</body> </html>'''
  return end_template



def header_template(team:str, color:str, team_image):
  team_template= f'''<div style="{color}; margin-top: 0px;  text-align: center; border: 3px solid green;">
              
                        <div class="head-content fws-hide player-head text-center">
                            <div >
                                <div class="head-title">
                                    <h2  style="color: rgb(255, 255, 255)" class="title ta-c">{team}</h2>
                                </div>
                            </div>
                            <div class="bottom-row" >
                                <div class="img-container" style="float:inherit; display:inline-block">
                                    <div class="ib posr">
                                                            <div class="img-wrapper shield no-circle">
                                            <img loading="lazy" src="{team_image}" alt="{team}">
                                        </div>
                                    </div>
                                </div>
                            </div>
                        </div>
        </div>
        
        '''
  return team_template



def team_description(desc:str,time:str):
  team_template=f'''<div style="text-align: center; margin-right: 10px; float:left">
            <div style="display: inline-block; flex:1; text-align: left;">
              </br>
              <h2>{desc}</h2>
              <strong><h1 style="text-align: center;"> {time}</h1></strong>
            </div>
        </div>
    '''
  return team_template  

def football_template():return "<h1> Football Teams </h1> <br>"

## Football

In [ ]:
def football_team(Team = str) :
  # Create an URL object
  url = f'https://www.besoccer.com/team/matches/{Team}'
  # Create object page
  page = requests.get(url)
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  soup = BeautifulSoup(page.text, 'lxml')
  team = soup.find('h2', class_ = "title ta-c").text
  next_games = soup.find("div", id="mod_detail_team_matches_on")
  games_json  = (next_games.find("script", {"type":"application/ld+json"}).contents)

  #time is not in the json, we pull it out from the raw html
  game = next_games.find("a", class_="match-link")
  hour = game.find("p").text
  opponent = game.find("")
  #Json contains all needed information about the match
  match_json = json.loads(str(games_json[0]).replace("\n",""))

  ### Select all fields 
  event_name = match_json['name']
  event_description = match_json['description']
  # Team Image
  if match_json["awayTeam"]["name"] == team:
    team_image = match_json["awayTeam"]['image'] 
    team_name = match_json["awayTeam"]["name"]
    opponent_name =  match_json["homeTeam"]["name"]
  else:
    team_image = match_json["homeTeam"]['image']
    team_name = match_json["homeTeam"]["name"]
    opponent_name =  match_json["awayTeam"]["name"]

  ### Parse dates
  #use datetime to select a more human redable date 
  event_time = dt.strptime(match_json['startDate'], '%Y-%m-%d')
  event_days = (event_time.date() - dt.today().date()).days
  event_time_text =  f'in {event_days} days '
  if  event_days < 2 : 
    event_time_text = 'tomorrow'
  elif event_days < 1:
    event_time_text = 'today'

  event_daytime = event_time_text + f' at {hour}'
  event_text = f' {team}' + ' plays ' + event_description + ' ' + event_daytime
  team_html = team_description(event_description,event_daytime)  
  ### Classification Table 
  # We obtain this from the BeScccer webiset of the team
  # Create an URL object
  url = f'https://www.besoccer.com/team/{Team}'
  # Create object page
  page = requests.get(url)
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  soup = BeautifulSoup(page.text, 'lxml')
  bckg_color = soup.find("div",class_= "head-info fixed-w-scroll color-light ")['style']
  classification = soup.find('div', id="mod_team_league_perf")
  class_table = classification.find('table')
  class_table.find_all('tr')[1].find_all('td')[1].find('img')['src']
  # Obtain every title of columns with tag <th>
  team_dics= {}
  images= []
  headers = ['Position', '']
  for i in class_table.find_all("th"):
    headers.append(i.text)

  mydata = pd.DataFrame(columns = headers)
  mydata
  # Create a for loop to fill mydata
  for j in class_table.find_all("tr")[1:]:
    row_data = j.find_all("td")
    row = [i.text.replace("\n","") for i in row_data]
    length = len(mydata)
    image = row_data[1].find('img')['src']
    images.append(image)
    mydata.loc[length] = row
  mydata = mydata.drop(columns = "")
  mydata['Team'] = images
  mydata.insert(2,'Team',mydata.pop('Team'))
  mydata.reset_index(inplace=True, drop=True)
  # Render Team
  header_html = header_template(team, bckg_color, team_image)
  def to_img_tag(path):
      return '<img   src="'+ path + '" width="50" >'
  table_html = mydata.to_html(escape=False,formatters=dict(Team=to_img_tag))
  file = open("sample.html","a")
  file.write(header_html)
  file.write(team_html)
  file.write(table_html)
  file.close()
  return None


name in lowcase low and separated by -
example: 
- Real Madrid = real-madrid
- Manchester United = manchester-united-fc

In [ ]:

football_team("real-madrid")
football_team("real-murcia")

## F1
> Ergast will be our source tables

In [ ]:
def F1():
  file = open("sample.html","a")
  file.write(f1_header_template())
  file.close()
  url = f'http://ergast.com/api/f1/current.json'
    # Create object page
  page = requests.get(url)
  races_json = (json.loads(page.text))
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  races = races_json['MRData']['RaceTable']['Races']
  df = pd.DataFrame(races)
  #df =pd.concat([df.drop(['time','Circuit'], axis=1), df['time'].apply(pd.Series), df['Circuit']], axis=1)
  df = df[df.columns.drop(list(df.filter(regex='Practice')))].drop(columns=["Sprint","url", "season"])
  df["Date"]= pd.to_datetime(df["date"])
  df.set_index("Date", inplace=True)
  df_trunk = df.truncate(before = dt.now().date()).head(1)[::-1]
  df_trunk.reset_index(inplace= True)
  race_dict = df_trunk.to_dict()
  event_time = dt.strptime(df_trunk.date[0],"%Y-%m-%d").date()
  event_days = (event_time - dt.today().date()).days
  event_time_text =  f'in {event_days} days '
  if  event_days < 2 : 
    event_time_text = 'tomorrow'
  elif event_days < 1:
    event_time_text = 'today'

  event_description= f'''{race_dict['raceName'][0]} at {race_dict['Circuit'][0]['circuitName']} in  {race_dict['Circuit'][0]['Location']['locality']}, {race_dict['Circuit'][0]['Location']['country']}'''
  event_daytime = f''' Race {event_time_text} at {race_dict['time'][0]} '''
  f1_html = team_description(event_description,event_daytime)
  prev_round = int(race_dict['round'][0]) -1
  url = f'https://ergast.com/api/f1/current/{prev_round}/driverStandings.json'
    # Create object page
  page = requests.get(url)
  standings_json = (json.loads(page.text))
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  standings = standings_json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']
  df_standings = pd.DataFrame(standings)
  df_driver = df_standings.Driver.apply(pd.Series)[['code','givenName','familyName','nationality']]
  df_constr = df_standings.Constructors.explode().apply(pd.Series)['name']
  df_standings = df_standings.drop(columns=['Driver','Constructors'])
  df_standings =pd.concat([df_standings,df_driver,df_constr],axis= 1)
  file = open("sample.html","a")
  file.write(f1_html)
  file.write(df_standings.to_html())
  file.close()
  return None

## Tennis

In [ ]:
def tennis_player(url_player:str):
  url = url_player
  # Create object page
  page = requests.get(url)
  headers = []
  # parser-lxml = Change html to Python friendly format
  # Obtain page's information
  soup = BeautifulSoup(page.text, 'lxml')
  rank = soup.find('div',class_="player-bio").text.split("#")[1][0]
  #make_team_template
  name = soup.find('h1').text
  image = soup.find('div',class_="main-headshot").find("img")['src']
  color= "background-color:#cc0000"
  header_html = header_template(name, color, image)
  competition = soup.find('div',class_='game-details').text
  competition = competition.split(",")[0]
  tennis_TABLE=soup.find_all('table')[2]
  for i in tennis_TABLE.find_all("th"):
    headers.append(i.text)
  mydata = pd.DataFrame(columns = headers)
  
  #Create a for loop to fill mydata
  a = 0
  for j in tennis_TABLE.find_all("tr")[1:]:
      a+=1
      if a>1: # We need to skip the row Men's Singles 
        row_data = j.find_all("td")
        row = [i.text.replace("\n","") for i in row_data]
        length = len(mydata)
        mydata.loc[length] = row
      else:
        race = j.find("td").text
  mydata['Race'] = race
  opponent = mydata['OPPONENT'].values[-1]
  time = mydata['SCORE'].values[-1]
  lost = bool(re.search('-', time))
  if lost:
    play = "lost"
  else:
    play= "plays"
  round= mydata['ROUND'].values[-1]
  player_html = f'<h1><strong>{name} stays at Rank {rank} and {play} the {competition} ,round:{round} against {opponent} on {time}</strong></h1>'
  file = open("sample.html","a")
  file.write(header_html)
  file.write(player_html)
  file.close()
  return None


## Main
> HTML Final Format

In [ ]:

# teams = ['real-murcia','real-madrid']
# tennis_players = ["http://www.espn.com/tennis/player/results/_/id/3782/carlos-alcaraz","http://www.espn.com/tennis/player/results/_/id/261/rafael-nadal"]
def main (user:dict):
  teams = user['teams']
  tennis_players = user['tennis']
  f1_bool = user['f1']

  file = open("sample.html","w")
  start_template()
  file.close()

  page = requests.get(tennis_players[0])

  # Football
  file = open("sample.html","a")
  file.write(football_template())
  file.close()
  for team in teams:
    football_team(team)

  for player in range(len(tennis_players)):
    tennis_player(tennis_players[player])
  if f1_bool:
    F1()

  file = open("sample.html","a")
  file.write(end_template())
  file.close()
  return HTML("sample.html")

dic = {'teams':['real-madrid','real-murcia'], 'tennis':["http://www.espn.com/tennis/player/results/_/id/3782/carlos-alcaraz","http://www.espn.com/tennis/player/results/_/id/261/rafael-nadal"], 'f1':True}
main(dic)




,Position,Matchday 5,Team,PTS,MP,W,D,L,GF,GA,GD
0,1,Real Madrid,,12,4,4,0,0,11,4,7
1,2,Barcelona,,10,4,3,1,0,11,1,10
2,3,Villarreal,,10,4,3,1,0,9,0,9
3,4,Real Betis,,9,4,3,0,1,7,3,4
4,5,Osasuna,,9,4,3,0,1,6,3,3
,Position,Matchday 3,Team,PTS,MP,W,D,L,GF,GA,GD
0,2,Osasuna Promesas,,4,2,1,1,0,3,1,2
1,3,R. Sociedad B,,4,2,1,1,0,3,1,2
2,4,Real Murcia,,4,2,1,1,0,3,1,2
3,5,Real Unión Club,,4,2,1,1,0,3,1,2
